In [1]:
import os
print(os.getcwd())  # Affiche le dossier courant


c:\Users\admin\Desktop\Projet\Baroclimat


In [ ]:
import datetime as dt
import json

from azureml.core import Workspace, Dataset, Datastore
import pandas as pd
import torch
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

from First_classif.dataset.text_dataset import CustomDataset
from First_classif.utils.storage_connector import StorageConnector
from First_classif.modeling.camembert_climate import ClimateCamembert


In [5]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\admin\AppData\Roaming\nltk_data...
[

True

In [ ]:
config_path = os.path.join("src", "config", "config.json")
with open(config_path, 'r') as f:
    config = json.load(f)  # Charger le JSON correctement

# Extraire le token Hugging Face
AZURE_STORAGE_CONNECTION_STRING = config.get("azure", {}).get("AZURE_STORAGE_CONNECTION_STRING", "").strip()

if not AZURE_STORAGE_CONNECTION_STRING:
    raise ValueError("AZURE_STORAGE_CONNECTION_STRING non défini dans le fichier config.json")

print("AZURE_STORAGE_CONNECTION_STRING récupéré avec succès.")

In [ ]:
# From Config File
BATCH_SIZE = 16
DEFAULT_MODEL_ID = "2024_03_24_14_08_15"
DEFAULT_EMBED_MODEL_ID = "cc.fr.300"
MAX_LEN = 512
STORAGE_CONTAINER_MODELS = "models"

# Global variable 
path_climate_topics = 'src/baroclimat/config/climate_topics_baro.toml'

# Inference parameter
path_input_dataset = 'articles/articles_gnews_barometre.csv'
model_id = DEFAULT_MODEL_ID
embed_model_id = DEFAULT_EMBED_MODEL_ID
col_name_text = "Cleaned_Text"
col_name_label = ""
evaluate_classifier = False
max_len = MAX_LEN
batch_size = BATCH_SIZE
local_save = False 

In [ ]:
config_path = os.path.join("src", "config", "config.json")
with open(config_path, 'r') as f:
    config = json.load(f)  # Charger le JSON correctement

# Extraire le token Hugging Face
subscription_id = config.get("azure", {}).get("subscription_id", "").strip()

if not subscription_id:
    raise ValueError("subscription_id non défini dans le fichier config.json")

print("subscription_id récupéré avec succès.")

In [ ]:
resource_group = 'RG-BAROMETRE-CENTRAL'
workspace_name = 'aml-barometre-weu'

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = Datastore.get(workspace, "raw_data")
if os.path.splitext(path_input_dataset)[-1] == '.parquet' :
    dataset = Dataset.Tabular.from_parquet_files(path=(datastore, path_input_dataset))
else :
    dataset = Dataset.Tabular.from_delimited_files(path=(datastore, path_input_dataset))
input_dataset = dataset.to_pandas_dataframe()

if input_dataset[col_name_text].isnull().sum() > 0:
    print(f"Found {input_dataset[col_name_text].isnull().sum()} empty text rows. Removing them.")
    input_dataset = input_dataset.dropna(subset=[col_name_text])

input_dataset = input_dataset.reset_index(drop=True)
# input_dataset = input_dataset.sample(frac=0.1, ignore_index=True)
input_dataset

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
Found 1 empty text rows. Removing them.


,Column1,Title,Websites,URL,Publication Date,Grouped_Subject,Subject,Cleaned_Text
0,0,"""En France, les transports sont responsables d...",franceinfo,https://www.francetvinfo.fr/replay-radio/zero-...,2024-01-13 08:00:00,Causes,Emissions de gaz à effet de serre (transport…),Cleaned_Text
1,1,Le secteur des transports est-il le seul à ne ...,franceinfo,https://www.francetvinfo.fr/replay-radio/le-vr...,2023-08-07 07:00:00,Causes,Emissions de gaz à effet de serre (transport…),Le secteur des transports est-il le seul à ne ...
2,2,Le plus grand volcan de la planète Mars était ...,franceinfo,https://www.francetvinfo.fr/replay-radio/le-bi...,2023-08-03 07:00:00,Causes,Emissions de gaz à effet de serre (transport…),"C’est un volcan géant, le plus grand de tout l..."
3,3,Transports : des véhicules électriques adaptés...,franceinfo,https://www.francetvinfo.fr/economie/transport...,2023-12-02 08:00:00,Causes,Emissions de gaz à effet de serre (transport…),Transports : des véhicules électriques adaptés...
4,4,L'impact de l'irrigation sur le système terres...,franceinfo,https://www.francetvinfo.fr/replay-radio/un-de...,2023-06-24 07:00:00,Causes,Elevage et utilisation des terres,L'impact de l'irrigation sur le système terres...
...,...,...,...,...,...,...,...,...
1547,1548,Morfo lève 4 millions d'euros pour internation...,JDN,https://www.journaldunet.com/iot/1517517-morfo...,2022-12-08 08:00:00,Solutions,Reforestation,Morfo lève 4 millions d'euros pour internatio...
1548,1549,Déforestation : état des lieux 2022,Gerbeaud.com,https://www.gerbeaud.com/nature-environnement/...,2023-01-02 08:00:00,Solutions,Reforestation,Déforestation : état des lieux en 2022 Une pri...
1549,1550,Journée internationale de l'arbre : les jeunes...,Cameroon Tribune,https://www.cameroon-tribune.cm/article.html/5...,2023-03-22 07:00:00,Solutions,Reforestation,Journée internationale de l’arbre : les jeunes...
1550,1551,Environnement - Bientôt deux microforêts en Eu...,L'Echo Républicain,https://www.lechorepublicain.fr/gasville-oisem...,2021-07-05 07:00:00,Solutions,Reforestation,"Bientôt deux microforêts en Eure-et-Loir, impl..."


In [9]:
print("Loading pretrained model and embedding model, this may take a while.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
storage_connector = StorageConnector(AZURE_STORAGE_CONNECTION_STRING, STORAGE_CONTAINER_MODELS)

encoder = AutoTokenizer.from_pretrained("camembert-base")
classifier = ClimateCamembert(tokenizer=encoder)
classifier.from_pretrained(model_id, device, storage_connector)

Loading pretrained model and embedding model, this may take a while.
Device: cpu


2025-04-03 10:04:34.020 | INFO     | First_classif.modeling.camembert_climate:from_pretrained:58 - Model not found locally. Loading it from Azure Blob Storage.


In [10]:
# Process input text
if not evaluate_classifier:
    col_name_label = None
inference_dataset = CustomDataset(input_dataset, encoder, max_len, col_name_label, col_name_text)
inference_loader = DataLoader(inference_dataset, batch_size=batch_size, shuffle=False)


In [11]:
# Perform inference
time = dt.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
print("Performing first step of inference: Climate change detection.")
y_pred, y_pred_probas, top_words = classifier.predict(inference_loader, device)
input_dataset["climate_related"] = y_pred
input_dataset["confidence"] = y_pred_probas
input_dataset["top_words"] = top_words

Performing first step of inference: Climate change detection.


CamembertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [12]:
input_dataset.head(10)

,Column1,Title,Websites,URL,Publication Date,Grouped_Subject,Subject,Cleaned_Text,climate_related,confidence,top_words
0,0,"""En France, les transports sont responsables d...",franceinfo,https://www.francetvinfo.fr/replay-radio/zero-...,2024-01-13 08:00:00,Causes,Emissions de gaz à effet de serre (transport…),Cleaned_Text,0,0.882788,"[an, C, ed]"
1,1,Le secteur des transports est-il le seul à ne ...,franceinfo,https://www.francetvinfo.fr/replay-radio/le-vr...,2023-08-07 07:00:00,Causes,Emissions de gaz à effet de serre (transport…),Le secteur des transports est-il le seul à ne ...,1,0.983167,"[environnement, pollution, climat]"
2,2,Le plus grand volcan de la planète Mars était ...,franceinfo,https://www.francetvinfo.fr/replay-radio/le-bi...,2023-08-03 07:00:00,Causes,Emissions de gaz à effet de serre (transport…),"C’est un volcan géant, le plus grand de tout l...",0,0.884879,"[eau, évolution, eau]"
3,3,Transports : des véhicules électriques adaptés...,franceinfo,https://www.francetvinfo.fr/economie/transport...,2023-12-02 08:00:00,Causes,Emissions de gaz à effet de serre (transport…),Transports : des véhicules électriques adaptés...,1,0.978892,"[électriques, électrique, serre]"
4,4,L'impact de l'irrigation sur le système terres...,franceinfo,https://www.francetvinfo.fr/replay-radio/un-de...,2023-06-24 07:00:00,Causes,Elevage et utilisation des terres,L'impact de l'irrigation sur le système terres...,1,0.981162,"[sécheresse, durable, sécheresse]"
5,5,Agriculture : changer les habitudes face au ré...,franceinfo,https://www.francetvinfo.fr/economie/emploi/me...,2019-08-08 07:00:00,Causes,Elevage et utilisation des terres,Agriculture : changer les habitudes face au ré...,1,0.982691,"[climatique, serre, réchauffement]"
6,6,Énergie : attention à la surconsommation élect...,franceinfo,https://www.francetvinfo.fr/economie/energie/e...,2023-06-20 07:00:00,Causes,Surconsommation,Énergie : attention à la surconsommation élect...,1,0.974965,"[consommation, chaleur, électrique]"
7,7,Sécheresse : la surconsommation d'eau des mill...,franceinfo,https://www.francetvinfo.fr/meteo/secheresse/s...,2023-05-12 07:00:00,Causes,Surconsommation,Sécheresse : la surconsommation d’eau des mil...,1,0.981929,"[sécheresse, eau, cher]"
8,8,Déforestation : l'inquiétude des producteurs d...,franceinfo,https://www.francetvinfo.fr/monde/environnemen...,2023-12-12 08:00:00,Causes,Déforestation,Déforestation : l'inquiétude des producteurs d...,1,0.977826,"[déforestation, déforestation, for]"
9,9,Environnement : malgré des engagements interna...,franceinfo,https://www.francetvinfo.fr/replay-radio/le-bi...,2023-10-25 07:00:00,Causes,Déforestation,Environnement : malgré des engagements interna...,1,0.981416,"[déforestation, déforestation, Environnement]"
